In [ ]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

Data preparation
Check if the missing values are presented in the features.
If there are missing values:
For caterogiral features, replace them with 'NA'
For numerical features, replace with with 0.0

In [57]:
import pandas as pd

In [58]:
df = pd.read_csv("course_lead_scoring.csv")
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [59]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [60]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [61]:
df['lead_source'] = df['lead_source'].fillna('NA')
df['industry'] = df['industry'].fillna('NA')
df['employment_status'] = df['employment_status'].fillna('NA')
df['location'] = df['location'].fillna('NA')

df['annual_income'] = df['annual_income'].fillna(0)

Question 1
What is the most frequent observation (mode) for the column industry?

NA
technology
healthcare
retail

In [62]:
df['industry'].mode()

0    retail
Name: industry, dtype: object

Question 2
Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

interaction_count and lead_score
number_of_courses_viewed and lead_score
number_of_courses_viewed and interaction_count
annual_income and interaction_count

In [63]:
abs(df['interaction_count'].corr(df['lead_score']))

np.float64(0.0098881824969131)

In [64]:
abs(df['number_of_courses_viewed'].corr(df['lead_score']))

np.float64(0.004878998354681263)

In [65]:
abs(df['number_of_courses_viewed'].corr(df['interaction_count']))

np.float64(0.023565222882888103)

In [66]:
abs(df['annual_income'].corr(df['interaction_count']))

np.float64(0.027036472404814327)

Split the data
Split your data in train/val/test sets with 60%/20%/20% distribution.
Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
Make sure that the target value y is not in your dataframe.

In [67]:
from sklearn.model_selection import train_test_split

In [68]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train, test_size=0.25, random_state=42)

In [69]:
y_train = df_train['converted'].values
del df_train['converted']

y_val = df_val['converted'].values
del df_val['converted']

y_test = df_test['converted'].values
del df_test['converted']

Question 3
Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
Round the scores to 2 decimals using round(score, 2).
Which of these variables has the biggest mutual information score?

industry
location
lead_source
employment_status

In [70]:
from sklearn.metrics import mutual_info_score, accuracy_score

In [71]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']
numerical = ['number_of_courses_viewed',
             'annual_income', 'interaction_count', 'lead_score']
label = 'converted'

In [72]:
def mutual_info_churn_score(series):
    return round(mutual_info_score(series, y_train), 2)


mis = df_train[categorical].apply(mutual_info_churn_score)
mis = mis.sort_values(ascending=False)
mis

lead_source          0.04
industry             0.01
employment_status    0.01
location             0.00
dtype: float64

Question 4
Now let's train a logistic regression.
Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
Fit the model on the training dataset.
To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [73]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [74]:
dv = DictVectorizer(sparse=False)

train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [75]:
model = LogisticRegression(solver='liblinear', C=1.0,
                           max_iter=1000, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)
accuracy_score(y_val, y_pred)

0.6996587030716723

Question 5
Let's find the least useful feature using the feature elimination technique.
Train a model using the same features and parameters as in Q4 (without rounding).
Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
Which of following feature has the smallest difference?

'industry'
'employment_status'
'lead_score'

In [76]:
features = ['industry', 'employment_status', 'lead_score']

for feature in features:
    columns = df_train.columns.to_list()
    columns.remove(feature)
    train_dict = df_train[columns].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    val_dict = df_val[columns].to_dict(orient='records')
    X_val = dv.transform(val_dict)

    model = LogisticRegression(
        solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    print(f"Without {feature}: {accuracy_score(y_val, y_pred)}")

Without industry: 0.6996587030716723
Without employment_status: 0.6962457337883959
Without lead_score: 0.7064846416382252


Question 6
Now let's train a regularized logistic regression.
Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
Train models using all the features as in Q4.
Calculate the accuracy on the validation dataset and round it to 3 decimal digits.
Which of these C leads to the best accuracy on the validation set?

0.01
0.1
1
10

In [77]:
train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

c_values = [0.01, 0.1, 1, 10, 100]
for c in c_values:
    model = LogisticRegression(
        solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    print(f"C={c}: {accuracy_score(y_val, y_pred)}")

C=0.01: 0.6996587030716723
C=0.1: 0.6996587030716723
C=1: 0.6996587030716723
C=10: 0.6996587030716723
C=100: 0.6996587030716723
